# LangChain Part 3.2: Output Parsing
- Important when using agents with tools e.g. calculator , search engine 
- Many systems expect data in specific way

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama
from os import getenv
from dotenv import load_dotenv
from pprint import pprint

load_dotenv()
# llm = ChatOpenAI(
#     model="gpt-4o-mini",
#     temperature=0.1,
#     max_tokens=512
# )
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base="https://openrouter.ai/api/v1",
    model="microsoft/phi-3-medium-128k-instruct:free",
    temperature=0.1,
)
# llm = ChatOllama(model='llama3.2', temperature=0.2, max_tokens=512)

result = llm.invoke("Hello, how are you today?")
print(result.content)

 I'm just a computer program, so I don't have feelings, but I'm ready to assist you! How can I help you today?


# Pydantic Model Parser
- Allows you to parse data into a Pydantic model

In [72]:
from pprint import pp
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

# Define your desired data structure.
class Movie(BaseModel):
    title: str = Field(description="Title of the movie")
    year: int = Field(description="Release year of the movie")
    director: str = Field(description="Director of the movie")

# And a query intended to prompt a language model to populate the data structure.
movie_query = "Provide details of the movie 'Inception'."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Movie)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

pp(prompt.dict())

{'name': None,
 'input_variables': ['query'],
 'optional_variables': [],
 'output_parser': None,
 'partial_variables': {'format_instructions': 'The output should be formatted '
                                              'as a JSON instance that '
                                              'conforms to the JSON schema '
                                              'below.\n'
                                              '\n'
                                              'As an example, for the schema '
                                              '{"properties": {"foo": '
                                              '{"title": "Foo", "description": '
                                              '"a list of strings", "type": '
                                              '"array", "items": {"type": '
                                              '"string"}}}, "required": '
                                              '["foo"]}\n'
                                              

In [73]:
chain = prompt | llm | parser

result = chain.invoke({"query": movie_query})
print(result)

title='Inception' year=2010 director='Christopher Nolan'


# YAML Parser
- Allows you to parse YAML data into a dictionary

In [80]:
from typing import List

from langchain.output_parsers import YamlOutputParser
from pydantic import BaseModel, Field

# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")
    

# And a query intented to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = YamlOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

chain.invoke({"query": joke_query})

Joke(setup="Why don't scientists trust atoms?", punchline='Because they make up everything!')

# Output Fixing Parser
- Allows you to fix the output of a model

## Auto-Fix Parser

In [81]:
from langchain.output_parsers import OutputFixingParser
from langchain.output_parsers import DatetimeOutputParser

output_parser = DatetimeOutputParser()

misformatted = "2022-01-01T00:00:00.000Z"

result = output_parser.invoke(misformatted)

print(result)



2022-01-01 00:00:00


In [82]:
new_parser = OutputFixingParser.from_llm(parser=output_parser, llm=llm)

In [83]:
new_parser.parse(misformatted)


datetime.datetime(2022, 1, 1, 0, 0)